In [1]:
from openai import OpenAI
from IPython.display import Markdown, display

class Conversation:
    def __init__(self, model):
        self.client = OpenAI()
        self.model = model
        self.messages = []
        
    def say(self, message, hidden=False):
        self.messages.append(
            {
                "role": "user",
                "content": message
            }
        )
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            max_tokens=3000,
            temperature=0.7,
        )
        # if response.choices[0].message.role == "assistant":
        self.messages.append(response.choices[0].message)
        if not hidden:
            display(Markdown(response.choices[0].message.content))


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi # pip install youtube-transcript-api

class YouTubeConversation(Conversation):
    def __init__(self, model, video_id):
        super().__init__(model)
        self.messages = [
            {
                "role": "system",
                "content": f"""
You are a video transcript summarizer. The video ID is {video_id}.

Here is the video transcript which includes the timestamp in seconds (pay special attention to timestamps because you'll be asked about them later).

---
{YouTubeConversation._get_transcript(video_id)}
---

In subsequent conversation, if someone askes about a URL to a point in the video then use the standard YouTube URL format with the video ID and the timestamp in seconds, e.g., https://www.youtube.com/watch?v={video_id}&t=1234s.
"""
            }
        ]

        self.say(
            """\
Create a quick summary of the video transcript above as a bulleted list of topics covered in time order. For each bullet, include the following:
- the topic
- a very terse summary of the topic (one sentence)
- the timestamp (seconds, just like above) of the first occurrence of that topic along with the line of transcript that introduces the topic
- the timestamp (seconds) of the most important part of the topic along with the line of transcript that introduces the topic

This summary serves as a quick reference for later, but shouldn't influence the formal summary you'll be asked to write later which will be more detailed and prose-like.
""",
            hidden=True,
        )
        
    @classmethod
    def _get_transcript(cls, video_id):
        data = YouTubeTranscriptApi.get_transcript(video_id)
        transcript = []
        for entry in data:
            transcript.append(f"{entry['start']}: {entry['text']}")
        return "\n".join(transcript)
    
    def start(self):
        self.say(f"Provide a formal summary of the video transcript. Start with an introduction and then provide a summary of the topics covered in the video. This time write it as well-formatted markdown prose that is more comfortable to read. You can make use of markdown sections and subsections or bulleted lists - whichever is more appropriate for capturing the flow of the transcript. At appropriate points in the prose create YouTube URL links to the starting point of the segment you're discussing. These links do not have to include a quote from the transcript, but should be relevant to the topic being discussed. If you do decide to include a quote, then correct spelling and punctuation (the original transcript was all lowercase and had no punctuation).")


class YouTubeConversation2(Conversation):
    def __init__(self, model, video_id):
        super().__init__(model)
        self.video_id = video_id
        self.messages = [
            {
                "role": "system",
                "content": f"""
You are an advanced AI assistant specialized in summarizing YouTube video transcripts. Your task is to provide comprehensive, well-structured summaries that capture the essence of the video content. The video ID you're working with is {video_id}.

Here is the complete video transcript, including timestamps in seconds:

---
{YouTubeConversation._get_transcript(video_id)}
---

Instructions for summarization:

1. Quick Overview:
   - Create a brief (2-3 sentences) overview of the video's main topic and purpose.

2. Detailed Summary:
   - Provide a detailed summary of the video content in well-formatted markdown prose.
   - Use appropriate headings (##, ###) to structure the summary logically.
   - Include 3-5 main sections, each covering a major topic or theme from the video.
   - Within each section, use subsections or bullet points as needed to break down subtopics.

3. Key Points and Timestamps:
   - For each main section, identify 2-3 key points or moments.
   - Create YouTube URL links for these key points using the format: https://www.youtube.com/watch?v={video_id}&t=X, where X is the timestamp in seconds.
   - Briefly explain why each highlighted moment is significant.

4. Notable Quotes:
   - Include 2-3 notable quotes from the transcript, correcting spelling and punctuation as needed.
   - Provide context for each quote and explain its relevance.

5. Visual Elements (if mentioned in the transcript):
   - Note any significant visual elements described in the transcript (e.g., charts, demonstrations, or visual aids).
   - Explain how these visuals contribute to the video's message.

6. Conclusion:
   - Summarize the main takeaways from the video in 2-3 sentences.
   - If applicable, mention any call-to-action or next steps suggested in the video.

7. Metadata (at the end of the summary):
   - Video Length: [HH:MM:SS] (calculate from the last timestamp)
   - Main Topics: List 3-5 main topics covered
   - Target Audience: Infer the likely target audience based on the content

Remember to maintain a neutral tone and focus on accurately representing the video's content. Your summary should be informative and easy to navigate, allowing readers to quickly understand the video's content and locate specific information if needed.
"""
            }
        ]

    @classmethod
    def _get_transcript(cls, video_id):
        data = YouTubeTranscriptApi.get_transcript(video_id)
        transcript = []
        for entry in data:
            transcript.append(f"{entry['start']}: {entry['text']}")
        return "\n".join(transcript)
    
    def start(self):
        self.say("Please provide a comprehensive summary of the YouTube video transcript based on the instructions given. Ensure your summary is well-structured, informative, and easy to navigate.")        

In [3]:
c = YouTubeConversation("gpt-4o", "D7BzTxVVMuw")
c.start()

# AI Engineer Summit 2025: Agent Engineering Day

The AI Engineer Summit 2025 focused on the theme of "Agent Engineering," bringing together leaders, builders, and experts in the field to discuss the current state, challenges, and future of AI agents. The event opened with a welcome address highlighting the transition from theoretical AI models to practical, real-world applications.

## Introduction and Welcome

The summit was introduced by the MCs, who set the stage for a day dedicated to builders and engineers. The event was streamed live to over 13,000 online viewers. [Watch the introduction](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=541s).

## Themes and Historical Context

The talks throughout the day covered various themes, including the historical development of AI agents from the 1950s to today, and the shift from theoretical models to practical applications. The discussion emphasized the importance of understanding the historical context to appreciate the current advancements and challenges in agent development. [Learn more about the themes](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=632s).

## Keynotes and Presentations

The summit featured a series of keynotes and presentations addressing the use of AI agents in finance, the challenges of deploying AI at scale, and practical examples of building effective AI agents. These sessions provided insights into real-world applications and the technical considerations involved in deploying AI agents. [Explore the presentations](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=694s).

## Practical Workshops and Sponsor Acknowledgements

Announcements were made about hands-on workshops scheduled for the following day, emphasizing the summit's focus on practical, actionable insights. The event also acknowledged the support of sponsors, highlighting the collaborative nature of the AI engineering community. [Details about workshops and sponsors](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=763s).

## Swix's Presentation on AI Engineering Trends

Swix, a key figure in the AI engineering community, delivered a presentation on the state of AI engineering, discussing the rise of AI agents and the skepticism surrounding them. He highlighted the trend towards agent-centric applications and the need for ongoing innovation and adaptability. [Watch Swix's presentation](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=925s).

## Defining Agents and Their Impact

A detailed discussion on the definition of AI agents was presented, exploring their significance and potential impact on various industries. This section aimed to clarify what constitutes an agent and the roles they can play in enhancing business processes. [Understand the definition of agents](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1259s).

## Capabilities and Model Evolution

The evolution of AI capabilities was examined, focusing on the diversity of models and how they are enabling more sophisticated agent functionalities. This segment highlighted the rapid advancements in AI technology and their implications for future developments. [Explore model evolution](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1363s).

## Agent Use Cases

The summit showcased various use cases for AI agents, from coding assistants to financial applications, illustrating the broad applicability and versatility of AI in solving complex problems. This section provided practical examples of how agents are being used today. [Discover agent use cases](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1472s).

## The Evolving Role of AI Engineers

The role of AI engineers is evolving towards building and optimizing agents, requiring new skill sets and approaches to software development. This section discussed the changing landscape of AI engineering and the new opportunities it presents. [Learn about the evolving role](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1588s).

## Conclusion

The summit concluded with a call to engage with speakers and sponsors, reinforcing the collaborative nature of the event. Attendees were encouraged to continue discussions in the Q&A lounges and at affiliated events, fostering a community of innovation and knowledge sharing. [Concluding remarks and next steps](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=6880s).

The AI Engineer Summit 2025 provided a comprehensive overview of the current state and future potential of AI agents, offering valuable insights and practical guidance for engineers and industry leaders. The event highlighted the importance of collaboration, innovation, and adaptability in the rapidly evolving field of AI engineering.

In [4]:
c.say("List all of the places where 'Agent' was defined in the transcript. Give me timestampped links.")

Here are the points in the video transcript where the term 'Agent' was defined or discussed, along with timestamped YouTube links:

1. **Initial Definition of Agents**  
   Swix provides a slide defining what an agent is, focusing on machine learning, AI, and software engineering perspectives.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1259s).

2. **Expanded Definition by Simon Willison**  
   An expanded definition of agents is discussed, featuring crowdsource input from Simon Willison.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1285s).

3. **OpenAI's New Definition of Agents**  
   The presentation mentions a new definition of agents released by OpenAI during the summit.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1340s).

In [5]:
c.say("No, list all of the definitions throught the transcript. This was only the first few minutes of the video.")


Certainly! Here is a list of all the places throughout the transcript where 'Agent' is defined or discussed, along with the corresponding timestamped links:

1. **Initial Definition by Swix**  
   Swix provides a slide defining what an agent is, focusing on machine learning, AI, and software engineering perspectives.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1259s).

2. **Expanded Definition by Simon Willison**  
   An expanded definition of agents is discussed, featuring crowdsource input from Simon Willison.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1285s).

3. **OpenAI's New Definition of Agents**  
   The presentation mentions a new definition of agents released by OpenAI during the summit.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=1340s).

4. **Anthropic's Take on Agents**  
   Barry Jang from Anthropic discusses their perspective on what constitutes an effective agent in their session.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=3749s).

5. **Sierra's Definition of Agents**  
   During the Sierra presentation, there's a discussion on how they conceptualize agents for their consumer brand products.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=4646s).

6. **Bloomberg's Agent Architecture**  
   The session on Bloomberg discusses how they define and use agents within their financial products and services.  
   [Watch here](https://www.youtube.com/watch?v=D7BzTxVVMuw&t=11457s).

These definitions and discussions provide various perspectives on what constitutes an agent and highlight the diversity of thought in the field.

In [6]:
c2.say("Come up with a Twitter post that make people want to watch the video. It should tug on a pain point from the video and link to a point in the video where I discuss it.")

🚀 Struggling to keep up with the rapid changes in AI? Discover how to harness the exponential growth in creativity and turn it into your superpower! 💡✨ Dive into the future of AI with John Berryman. Watch now: [https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235](https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235) #AI #Innovation #MachineLearning #Creativity

In [7]:
c2.say("That's not bad. Let's remove the emojis and hashtags, and try to make it as 2 posts. The first one should be a teaser with the problem, and the second should link out to the particular point in the video where I discuss it. Actually - do this first, create a few candidate ideas for the the hook (problem, engaging conversation point) and then for each write a doublep-post like I just said.")

Certainly! Here are a few candidate ideas for hooks, each followed by a double-post format:

### Candidate 1: The Challenge of Keeping Up
**Post 1 (Teaser):**  
Feeling overwhelmed by the fast-paced advancements in AI? It's a common struggle for many trying to stay ahead in this evolving landscape.

**Post 2 (Link):**  
Join John Berryman as he shares insights on leveraging the exponential growth in creativity to gain an edge. Watch here: [https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235](https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235)

---

### Candidate 2: Transforming Fear into Opportunity
**Post 1 (Teaser):**  
Does the rapid pace of AI development leave you anxious about the future? You're not alone—many feel the same pressure.

**Post 2 (Link):**  
Discover how to turn that fear into a creative advantage with John Berryman's strategies. Dive in: [https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235](https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235)

---

### Candidate 3: Mastering AI Tools
**Post 1 (Teaser):**  
Are AI tools leaving you feeling outmatched and underprepared? It's a familiar challenge in today's tech-driven world.

**Post 2 (Link):**  
Learn how to master these tools and enhance your creativity with insights from John Berryman. See more: [https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235](https://www.youtube.com/watch?v=60HAtHVBYj8&t=2235)

Please let me know if you need further adjustments or another set of ideas!

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
YouTubeTranscriptApi.get_transcript("YCrngJCkI8I")

[{'text': 'whoa simmer down there', 'start': 2.24, 'duration': 3.0},
 {'text': 'Sparky the hell did we just walk into',
  'start': 10.0,
  'duration': 5.599},
 {'text': "hey Uncle Jack I don't see Jesse Pinkman",
  'start': 13.28,
  'duration': 3.919},
 {'text': 'here he was right here', 'start': 15.599, 'duration': 3.76},
 {'text': 'before got eyes on', 'start': 17.199, 'duration': 4.121},
 {'text': 'Pinkman what do you figure you may be',
  'start': 19.359,
  'duration': 4.121},
 {'text': 'headed down that Gully', 'start': 21.32, 'duration': 6.279},
 {'text': "there that's the way I Go Frankie Lester",
  'start': 23.48,
  'duration': 7.52},
 {'text': 'go find him jack', 'start': 27.599, 'duration': 6.0},
 {'text': 'these guys are', 'start': 31.0, 'duration': 2.599},
 {'text': 'DEA no Jack Jack Jack Jack no no',
  'start': 40.559,
  'duration': 12.8},
 {'text': 'Jack Jack Jack Jack', 'start': 48.84, 'duration': 6.039},
 {'text': 'no', 'start': 53.359, 'duration': 5.401},
 {'text': "Ja